In [ ]:
import pandas as pd

# 读取 CSV 文件
file_path = '/home/yongkang/work/TableLlama/generate_prompt/test_semi_sotab-re-withunlabel.csv'
data = pd.read_csv(file_path)
data['label'] = data['label'].astype(int)


In [24]:
data.head()

,table_id,column_id,label,data
0,Product_beautyinstonejewelry.com_September2020...,6,0,2020-10-07; 2020-10-07; 2020-10-09; 2020-10-03...
1,Product_beautyinstonejewelry.com_September2020...,2,1,USD; USD; USD; USD; USD; USD; USD; USD; USD; U...
2,Product_beautyinstonejewelry.com_September2020...,5,2,Beauty In Stone Jewelry; Beauty In Stone Jewel...
3,Product_beautyinstonejewelry.com_September2020...,1,-1,"40\"" Chain Classic designer inspired famous fl..."
4,Product_beautyinstonejewelry.com_September2020...,3,-1,69; 49; 99; 65; 7; 30; 20; 59; 69; 80; 69; 99;...


In [18]:

import warnings
warnings.filterwarnings("ignore")

import argparse
import json
import math
import os
import random
from time import time
import mlflow
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from collections import defaultdict

# import pytrec_eval
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from accelerate import Accelerator
from copy import deepcopy

torch.backends.cuda.matmul.allow_tf32 = True

from accelerate import DistributedDataParallelKwargs
import wandb

from argparse import Namespace
import torch
import random
import pandas as pd
import numpy as np
import os
import pickle
import json
import re
import transformers
from torch.utils import data
from torch.nn.utils import rnn
from transformers import AutoTokenizer

from typing import List
from functools import reduce
import operator

from itertools import chain
import copy
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [1]:
from supervised_fine_tune import PROMPT_DICT
def generate_prompt(instruction, question, input_seg=None):
  if input:
    return PROMPT_DICT["prompt_input"].format(instruction=instruction, input_seg=input_seg, question=question)
  else:
    return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

/home/yongkang/miniconda3/envs/observatory/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
import math

config = transformers.AutoConfig.from_pretrained(
    "osunlp/TableLlama",
    cache_dir="./cache",
)

orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len and 8192 > orig_ctx_len:
    scaling_factor = float(math.ceil(8192 / orig_ctx_len))
    config.rope_scaling = {"type": "linear", "factor": scaling_factor}

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "osunlp/TableLlama",
    cache_dir="./cache",
    model_max_length=8192 if 8192 > orig_ctx_len else orig_ctx_len,
    # padding_side="right",
    padding_side="left",
    use_fast=False,
)


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


# TURL-CTA

In [17]:
# read in the type vocab
with open("/home/yongkang/work/TableLlama/generate_prompt/type_vocab_turl-cta.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}
types = list(type2id.keys())

all_label_names = ", ".join(id2type.values())

In [18]:
# read df
import os
import pandas as pd
filepath = "/home/yongkang/work/TableLlama/generate_prompt/turl_cta_test_all_with_metadata.csv"
df = pd.read_csv(filepath)
df['label'] = df['label'].apply(lambda x: eval(x))
index_list = df['label'].apply(lambda x: [i for i, value in enumerate(x) if value == 1])

label_names = []
for indices in index_list:
    labels = [id2type.get(idx) for idx in indices]
    label_names.append(labels)

# 将结果保存到 DataFrame 中（可选）
df['label'] = label_names

# 输出结果
print(df[:10])

     table_id                            pgTitle              secTitle  \
0   1001122-3                    elon university            greek life   
1   1001122-3                    elon university            greek life   
2   1001122-3                    elon university            greek life   
3  10039383-3  australian federal election, 1943  seats changing hands   
4  10039383-3  australian federal election, 1943  seats changing hands   
5  10039383-3  australian federal election, 1943  seats changing hands   
6  10039383-3  australian federal election, 1943  seats changing hands   
7  10039383-3  australian federal election, 1943  seats changing hands   
8  10039383-3  australian federal election, 1943  seats changing hands   
9  10039383-3  australian federal election, 1943  seats changing hands   

  caption                        headers  \
0     NaN        interfraternity council   
1     NaN  national pan-hellenic council   
2     NaN            panhellenic council   
3     NaN

In [19]:
from tqdm import tqdm
import os
import pandas as pd
import time, json

num_row = 3 # how many rows to include in the table prompt
k = 5
output_data = []

for tqdm_idx, group_df in tqdm(df.groupby('table_id')):
    group_df.sort_values(by=['column_index'], inplace=True)
    group_df.reset_index(drop=True, inplace=True)
    group_df['data'] = group_df['data'].astype(str)
    
    pgTitle = group_df['pgTitle'].iloc[0]
    secTitle = group_df['secTitle'].iloc[0]
    caption = group_df['caption'].iloc[0]
    wiki_info = f"[TLE] "
    if pgTitle != "nan":
        wiki_info += f"The Wikipedia page is about {pgTitle}. "
    if secTitle != "nan":
        wiki_info += f"The Wikipedia section is about {secTitle}. "
    if caption != "nan":
        wiki_info += f"The table caption is about {caption}. "

    col_values = []
    min_row_len = min([len(row['data'].split(";")) for row_idx, row in group_df.iterrows()])
    min_row_len = min(min_row_len, num_row)
    for column_index, col in group_df.iterrows():
        col_values.append(col['data'].split(";")[:min_row_len])
    # transpose the list
    col_values = list(map(list, zip(*col_values)))


    col_head = f"[TAB] col: | "
    for column_index in range(1, len(group_df) + 1):
        col_head += f"{group_df.iloc[column_index-1]['headers']} |"

    table_cell = ""
    for row_idx in range(1, min_row_len + 1):
        example_values = " | ".join(col_values[row_idx-1])  # Extract first 5 
        if row_idx == 1:
            table_cell += f"row {row_idx}: | {example_values} |"
        else:
            table_cell += f" [SEP] row {row_idx}: | {example_values} |"

    df_target = group_df[group_df['label'].apply(lambda x: len(x) != 0)]
    for idx in range(len(df_target)):
        column_index = df_target.iloc[idx]["column_index"]+1
        label = df_target.iloc[idx]["label"]
        header = df_target.iloc[idx]["headers"]
        entities = ", ".join([f"<{item.strip()}>" for item in df_target.iloc[idx]["data"].split(";")[:k]])

        table_prompt = {}
        table_prompt["table_id"] = str(tqdm_idx)
        table_prompt["instruction"] = "This is a column type annotation task. The goal for this task is to choose correct types for one selected column of the table from the given candidates. The Wikipedia page, section and table caption (if any) provide important information for choosing the correct column types."
        table_prompt["input"] = wiki_info + col_head + table_cell
        table_prompt["question"] = f"The column '{header}' contains following entities: {entities}, stc. The column type candidates are: {all_label_names}. What are the correct column types for this column (column name: {header}; entities: {entities}, etc)?"
        table_prompt["ground_truth"] = label
        table_prompt["entity"] = [item.strip() for item in df_target.iloc[idx]["data"].split(";")[:k]]
        table_prompt["output"] = ", ".join(label)
        table_prompt["input_seg"] = wiki_info + col_head + table_cell
        output_data.append(table_prompt)
    
        
output_file = './dataset/turl-cta.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

  0%|          | 0/4764 [00:00<?, ?it/s]

100%|██████████| 4764/4764 [00:11<00:00, 402.86it/s]


# TURL-CPA

In [20]:
# read in the type vocab
with open("./generate_prompt/type_vocab_turl-cpa.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}
types = list(type2id.keys())

all_label_names = ", ".join(id2type.values())
all_label_names

'baseball.baseball_player.batting_stats-baseball.batting_statistics.team, book.author.works_written, time.event.locations, film.film.genre, government.politician.party-government.political_party_tenure.party, people.person.place_of_birth, film.film.release_date_s-film.film_regional_release_date.film_release_region, american_football.football_player.position_s, cvg.computer_videogame.publisher, people.person.spouse_s-people.marriage.spouse, film.actor.film-film.performance.character, sports.sports_team.sport, base.biblioness.bibs_location.country, sports.sports_facility.home_venue_for-sports.team_venue_relationship.team, award.award_nominated_work.award_nominations-award.award_nomination.ceremony, film.film.directed_by, base.culturalevent.event.entity_involved, music.album.artist, sports.sports_team.location, sports.sports_position.players-sports.sports_team_roster.position, sports.pro_athlete.teams-sports.sports_team_roster.position, soccer.football_player.statistics-soccer.football_pl

In [21]:
# read df
import os
import pandas as pd
filepath = "/home/yongkang/work/TableLlama/generate_prompt/turl_cpa_test_all_with_metadata.csv"
df = pd.read_csv(filepath)

df['label'] = df['label'].apply(lambda x: eval(x))
index_list = df['label'].apply(lambda x: [i for i, value in enumerate(x) if value == 1])

label_names = []
for indices in index_list:
    labels = [id2type.get(idx) for idx in indices]
    label_names.append(labels)

# 将结果保存到 DataFrame 中（可选）
df['label'] = label_names

# 输出结果
print(df[:10])

     table_id                            pgTitle              secTitle  \
0  10039383-3  australian federal election, 1943  seats changing hands   
1  10039383-3  australian federal election, 1943  seats changing hands   
2  10039383-3  australian federal election, 1943  seats changing hands   
3  10039383-3  australian federal election, 1943  seats changing hands   
4  10039383-3  australian federal election, 1943  seats changing hands   
5  10039383-3  australian federal election, 1943  seats changing hands   
6  10039383-3  australian federal election, 1943  seats changing hands   
7  10039383-3  australian federal election, 1943  seats changing hands   
8  10039383-3  australian federal election, 1943  seats changing hands   
9  10039383-3  australian federal election, 1943  seats changing hands   

  caption      headers                                              label  \
0     NaN         seat                                                 []   
1     NaN     pre-1943         

In [22]:
from tqdm import tqdm
import os, json
import pandas as pd
import time

num_row = 3 # how many rows to include in the table prompt
k = 5
output_data = []

for tqdm_idx, group_df in tqdm(df.groupby('table_id')):
    group_df['column_index'] = group_df['column_index'].astype(int)
    group_df.sort_values(by=['column_index'], inplace=True)
    group_df.reset_index(drop=True, inplace=True)
    group_df['data'] = group_df['data'].astype(str)

    col_values = []
    min_row = min([len(row['data'].split(";")) for row_idx, row in group_df.iterrows()])
    min_row_len = min(min_row, num_row)
    for column_index, col in group_df.iterrows():
        col_values.append(col['data'].split(";")[:min_row_len])
    # transpose the list
    col_values = list(map(list, zip(*col_values)))


    col_head = f"[TAB] col: | "
    for column_index in range(1, len(group_df) + 1):
        col_head += f"{group_df.iloc[column_index-1]['headers']} |"

    table_cell = ""
    for row_idx in range(1, min_row_len + 1):
        example_values = " | ".join(col_values[row_idx-1])  # Extract first 5 
        if row_idx == 1:
            table_cell += f"row {row_idx}: | {example_values} |"
        else:
            table_cell += f" [SEP] row {row_idx}: | {example_values} |"

    df_target = group_df[group_df['label'].apply(lambda x: len(x) != 0)]
    for idx in range(len(df_target)):
        column_index = df_target.iloc[idx]["column_index"]+1
        label = df_target.iloc[idx]["label"]
        header = df_target.iloc[idx]["headers"]
        
        k = min(5, min_row)
        entity_pairs = ", ".join(
            f"<({group_df.iloc[0]['data'].split(';')[i]}), ({df_target.iloc[idx]['data'].split(';')[i]})>"
            for i in range(k)
        )

        table_prompt = {}
        table_prompt["table_id"] = str(tqdm_idx)
        table_prompt["instruction"] = "This is a relation extraction task. The goal for this task is to choose correct relations between two selected columns of the table from the given candidates. The Wikipedia page, section and table caption (if any) provide important information for choosing the correct relation types."
        table_prompt["question"] = f"The two selected column names are: <({group_df.iloc[0]['headers']}),({header})>. The entity pairs for these two columns are: {entity_pairs}, etc. The relation type candidates are: {all_label_names}. What are the correct relation type for the two selected columns (column name: <({group_df.iloc[0]['headers']}),({header})> entity pairs: {entity_pairs}, etc)?"
        table_prompt["input"] = col_head + table_cell
        table_prompt["ground_truth"] = label
        table_prompt["output"] = ", ".join(label)
        table_prompt["input_seg"] = col_head + table_cell
        output_data.append(table_prompt)
    
        
output_file = './dataset/turl-cpa.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

  0%|          | 0/1467 [00:00<?, ?it/s]

100%|██████████| 1467/1467 [00:07<00:00, 190.77it/s]


# SOTAB-CTA

In [19]:
# read in the type vocab
with open("/home/yongkang/work/TableLlama/generate_prompt/type_vocab_sotab-cta.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}
types = list(type2id.keys())

all_label_names = ", ".join(id2type.values())

In [37]:
# read df
import os
import pandas as pd
filepath = "./generate_prompt/test_semi_sotab_withunlabel.csv"
df = pd.read_csv(filepath)
df['label'] = df['label'].map(id2type).fillna(-1)
print(df[:10])

                                            table_id  column_index  \
0  Product_beautyinstonejewelry.com_September2020...             3   
1  Product_beautyinstonejewelry.com_September2020...             2   
2  Product_beautyinstonejewelry.com_September2020...             6   
3  Product_beautyinstonejewelry.com_September2020...             5   
4  Product_beautyinstonejewelry.com_September2020...             4   
5  Product_beautyinstonejewelry.com_September2020...             0   
6  Product_beautyinstonejewelry.com_September2020...             1   
7  Product_royalpoolandspa.com_September2020_CTA....             0   
8  Product_royalpoolandspa.com_September2020_CTA....             1   
9  Product_royalpoolandspa.com_September2020_CTA....             2   

              label                                               data  
0             price  69; 49; 99; 65; 7; 30; 20; 59; 69; 80; 69; 99;...  
1          currency  USD; USD; USD; USD; USD; USD; USD; USD; USD; U...  
2         

In [38]:
from tqdm import tqdm
import os
import pandas as pd
import time

num_row = 3 # how many rows to include in the table prompt
k = 5
output_data = []

for tqdm_idx, group_df in tqdm(df.groupby('table_id')):
    group_df.sort_values(by=['column_index'], inplace=True)
    group_df.reset_index(drop=True, inplace=True)
    group_df['data'] = group_df['data'].astype(str)

    col_values = []
    min_row_len = min([len(row['data'].split(";")) for row_idx, row in group_df.iterrows()])
    min_row_len = min(min_row_len, num_row)
    for column_index, col in group_df.iterrows():
        col_values.append(col['data'].split(";")[:min_row_len])
    # transpose the list
    col_values = list(map(list, zip(*col_values)))

    col_head = f"[TAB] col: | "
    for column_index in range(1, len(group_df) + 1):
        col_head += f"Column {column_index} | "

    table_cell = ""
    for row_idx in range(1, min_row_len + 1):
        example_values = " | ".join(col_values[row_idx-1])  # Extract first 5 
        if row_idx == 1:
            table_cell += f"row {row_idx}: | {example_values} |"
        else:
            table_cell += f" [SEP] row {row_idx}: | {example_values} |"

    df_target = group_df[group_df["label"] != -1]
    for idx in range(len(df_target)):
        column_index = df_target.iloc[idx]["column_index"]+1
        label = df_target.iloc[idx]["label"]
        entities = ", ".join([f"<{item.strip()}>" for item in df_target.iloc[idx]["data"].split(";")[:k]])

        table_prompt = {}
        table_prompt["table_id"] = str(tqdm_idx)
        table_prompt["instruction"] = "This is a column type annotation task. The goal for this task is to choose the correct type for one selected column of the table from the given candidates."
        table_prompt["input"] = col_head + table_cell
        table_prompt["question"] = f"The column 'Column {column_index}' contains following entities: {entities}, etc. The column type candidates are: {all_label_names}. What are the correct column type for this column (column name: Column {column_index}; entities: {entities}, etc)?"
        table_prompt["ground_truth"] = [label]
        table_prompt["entity"] = [item.strip() for item in df_target.iloc[idx]["data"].split(";")[:k]]
        table_prompt["output"] = label
        table_prompt["input_seg"] = col_head + table_cell
        output_data.append(table_prompt)
    
        
output_file = './dataset/sotab-cta.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

100%|██████████| 7026/7026 [00:15<00:00, 453.14it/s]


# SOTAB-CPA

In [20]:
# read in the type vocab
with open("./generate_prompt/type_vocab_sotab-cpa.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}

print(len(id2type))
all_label_names = ", ".join(id2type.values())
all_label_names

176


'priceValidUntil, priceCurrency, brand, productID, gtin13, sku, postalCode, telephone, itemCondition, availability, mpn, endDate, startDate, worstRating, addressLocality, addressRegion, reviewCount, bestRating, ratingValue, openingHours, longitude, latitude, description, performer, category, price, validFrom, url, image, weight, servingSize, fatContent, carboHydrateContent, proteinContent, sugarContent, saturatedFatContent, datePublished, prepTime, recipeYield, recipeIngredient, cookTime, recipeCategory, totalTime, keywords, recipeCuisine, recipeInstructions, calories, sodiumContent, releaseDate, manufacturer, color, model, organizer, addressCountry, workLocation, gtin8, location, eventStatus, review, email, transFatContent, cholesterolContent, fiberContent, birthDate, deathDate, inventoryLevel, opens, dayOfWeek, closes, contactType, priceRange, legalName, address, author, duration, byArtist, inAlbum, ratingCount, unsaturatedFatContent, gtin12, streetAddress, publisher, headline, text,

In [25]:
import os
import pandas as pd
filepath = "./generate_prompt/test_semi_sotab-re-withunlabel.csv"
df = pd.read_csv(filepath)
df['label'] = df['label'].map(id2type).fillna(-1)
print(df[df['table_id'] == 'Book_2d-market.com_September2020_CPA.json.gz'])

                                           table_id  column_id          label  \
2783   Book_2d-market.com_September2020_CPA.json.gz          8          price   
2784   Book_2d-market.com_September2020_CPA.json.gz          5  datePublished   
2785   Book_2d-market.com_September2020_CPA.json.gz         11  priceCurrency   
2786   Book_2d-market.com_September2020_CPA.json.gz          1    description   
2787   Book_2d-market.com_September2020_CPA.json.gz          2     bookFormat   
2788   Book_2d-market.com_September2020_CPA.json.gz          3  numberOfPages   
2789   Book_2d-market.com_September2020_CPA.json.gz          4          genre   
2790   Book_2d-market.com_September2020_CPA.json.gz          6       keywords   
2791   Book_2d-market.com_September2020_CPA.json.gz          7         author   
2792   Book_2d-market.com_September2020_CPA.json.gz          9   availability   
2793   Book_2d-market.com_September2020_CPA.json.gz         10            url   
54304  Book_2d-market.com_Se

In [26]:
from tqdm import tqdm
import os
import pandas as pd
import time

num_row = 3 # how many rows to include in the table prompt
k = 5
output_data = []

for tqdm_idx, group_df in tqdm(df.groupby('table_id')):
    group_df['column_id'] = group_df['column_id'].astype(int)
    group_df.sort_values(by=['column_id'], inplace=True)
    group_df.reset_index(drop=True, inplace=True)
    group_df['data'] = group_df['data'].astype(str)

    col_values = []
    min_row = min([len(row['data'].split(";")) for row_idx, row in group_df.iterrows()])
    min_row_len = min(min_row, num_row)
    for column_index, col in group_df.iterrows():
        col_values.append(col['data'].split(";")[:min_row_len])
    # transpose the list
    col_values = list(map(list, zip(*col_values)))

    col_head = f"[TAB] col: | "
    for column_index in range(1, len(group_df) + 1):
        col_head += f"Column {column_index} | "

    table_cell = ""
    for row_idx in range(1, min_row_len + 1):
        example_values = " | ".join(col_values[row_idx-1])  # Extract first 5 
        if row_idx == 1:
            table_cell += f"row {row_idx}: | {example_values} |"
        else:
            table_cell += f" [SEP] row {row_idx}: | {example_values} |"

    df_target = group_df[(group_df["label"] != -1)]
    for idx in range(len(df_target)):
        column_index = df_target.iloc[idx]["column_id"]+1
        label = df_target.iloc[idx]["label"]
        
        k = min(5, min_row)
        entity_pairs = ", ".join(
            f"<({group_df.iloc[0]['data'].split(';')[i]}), ({df_target.iloc[idx]['data'].split(';')[i]})>"
            for i in range(k)
        )

        table_prompt = {}
        table_prompt["table_id"] = str(tqdm_idx)
        table_prompt["instruction"] = "This is a relation extraction task. The goal for this task is to choose correct relation between two selected columns of the table from the given candidates."
        table_prompt["question"] = f"The two selected column names are: <(Column 1),(Column {column_index})>. The entity pairs for these two columns are: {entity_pairs}, etc. The relation type candidates are: {all_label_names}. What are the correct relation type for the two selected columns (column name: <(Column 1),(Column {column_index})>. entity pairs: {entity_pairs}, etc)?"
        table_prompt["input"] = col_head + table_cell
        table_prompt["ground_truth"] = [label]
        table_prompt["output"] = label
        table_prompt["input_seg"] = col_head + table_cell
        output_data.append(table_prompt)
    
        
output_file = './dataset/sotab-cpa.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

  0%|          | 0/6480 [00:00<?, ?it/s]

100%|██████████| 6480/6480 [00:30<00:00, 209.12it/s]


# Gittables

In [38]:
# read df
import os
import pandas as pd
git = 'sp'
cv = '0'

filepath = f"./generate_prompt/{git}_semi1_cv_{cv}.csv"
df = pd.read_csv(filepath)
with open("./generate_prompt/type2id_gitsc.pkl", "rb") as file: 
    type2id = pickle.load(file)
id2type = {int(v): k for k, v in type2id.items()}
types = list(type2id.keys())
all_label_names = ", ".join(types)
print(all_label_names)

identifier, name, description, title, value, status, latitude, longitude, comment, email, category, version, gender, address, location, price, amount, weight, startDate, productID, position, endDate, language, size, height, startTime, color, target, currency, author, birthDate, department, image, sku, parent, endTime, width, postalCode, elevation, model, area, keywords, device, creator, dateCreated, publisher, depth, releaseDate, permissionType, step, result, frequency, abstract


In [39]:
from tqdm import tqdm
import os
import pandas as pd
import time

num_row = 5 # how many rows to include in the table prompt
k = 10
output_data = []

for tqdm_idx, group_df in tqdm(df.groupby('table_id')):
    group_df['col_idx'] = group_df['col_idx'].astype(int)
    group_df['class_id'] = group_df['class_id'].astype(int)
    group_df.sort_values(by=['col_idx'], inplace=True)
    group_df.reset_index(drop=True, inplace=True)
    group_df['data'] = group_df['data'].astype(str)

    col_values = []
    min_row_len = min([len(row['data'].split(";")) for row_idx, row in group_df.iterrows()])
    min_row_len = min(min_row_len, num_row)
    for column_index, col in group_df.iterrows():
        col_values.append(col['data'].split(";")[:min_row_len])
    # transpose the list
    col_values = list(map(list, zip(*col_values)))

    col_head = f"[TAB] col: | "
    for column_index in range(1, len(group_df) + 1):
        col_head += f"Column {column_index} | "

    table_cell = ""
    for row_idx in range(1, min_row_len + 1):
        example_values = " | ".join(col_values[row_idx-1])  # Extract first 5 
        if row_idx == 1:
            table_cell += f"row {row_idx}: | {example_values} |"
        else:
            table_cell += f" [SEP] row {row_idx}: | {example_values} |"

    df_target = group_df[group_df["class_id"] > -1]
    for idx in range(len(df_target)):
        column_index = df_target.iloc[idx]["col_idx"]+1
        label = df_target.iloc[idx]["class"]
        entities = ", ".join([f"<{item.strip()}>" for item in df_target.iloc[idx]["data"].split(";")[:k]])

        table_prompt = {}
        table_prompt["table_id"] = str(tqdm_idx)
        table_prompt["instruction"] = "This is a column type annotation task. The goal for this task is to choose the correct type for one selected column of the table from the given candidates."
        table_prompt["input"] = col_head + table_cell
        table_prompt["question"] = f"The column 'Column {column_index}' contains following entities: {entities}, etc. The column type candidates are: {all_label_names}. What are the correct column type for this column (column name: Column {column_index}; entities: {entities}, etc)?"
        table_prompt["ground_truth"] = [label.split(":")[1]]
        table_prompt["entity"] = [item.strip() for item in df_target.iloc[idx]["data"].split(";")[:k]]
        table_prompt["output"] = label.split(":")[1]
        table_prompt["input_seg"] = col_head + table_cell
        output_data.append(table_prompt)
    
        
output_file = f'./dataset/gittables-{git}-{cv}.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

  0%|          | 0/574 [00:00<?, ?it/s]

100%|██████████| 574/574 [00:01<00:00, 436.10it/s]


# data analysis - sotab

In [12]:
from rapidfuzz import process, fuzz
import json
import pickle
from sklearn.metrics import confusion_matrix, f1_score

def match_closest_type(llm_output, valid_types):
    match, score, _ = process.extractOne(llm_output, valid_types, scorer=fuzz.ratio)
    return match   # Adjust threshold as needed

In [13]:
import statistics

micro_f1_scores = []
macro_f1_scores = []

task = "sotab-cpa"
pred_file = f"./output/{task}.json"
with open(f"./generate_prompt/type_vocab_{task}.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}
types = list(type2id.keys())

with open(pred_file, "r") as f:
    col_type = json.load(f)

ground_truth_list = []
pred_list = []
for i in range(len(col_type)):
    item = col_type[i]
    if item["predict"] == None:
        continue

    ground_truth = item["ground_truth"]
    # pred = item["predict"].strip("</s>").split(",")
    pred = item["predict"].split("</s>")[0].split(", ")
    ground_truth_list.append(ground_truth)
    pred_list.append(pred)
ts_true_list = [type2id[l[0]] for l in ground_truth_list]

ts_pred_list = []
for p in pred_list:
    pred_i = ', '.join(p)
    if pred_i in types:
        ts_pred_list.append(type2id[pred_i])
    else:
        ts_pred_list.append(type2id[match_closest_type(pred_i, types)])

ts_micro_f1 = f1_score(ts_true_list,
                    ts_pred_list,
                    average="micro")
ts_macro_f1 = f1_score(ts_true_list,
                    ts_pred_list,
                     average="macro")

micro_f1_scores.append(ts_micro_f1)
macro_f1_scores.append(ts_macro_f1)
print(f"ts_micro_f1={ts_micro_f1:.4f}, ts_macro_f1={ts_macro_f1:.4f}")


ts_micro_f1=0.1696, ts_macro_f1=0.1381


# data analysis - GitTables

In [1]:
from rapidfuzz import process, fuzz
import json
import pickle
from sklearn.metrics import confusion_matrix, f1_score

def match_closest_type(llm_output, valid_types):
    match, score, _ = process.extractOne(llm_output, valid_types, scorer=fuzz.ratio)
    return match   # Adjust threshold as needed

In [5]:
import statistics
import json
import pickle
from sklearn.metrics import f1_score

micro_f1_scores = []
macro_f1_scores = []

for cv in ["0", "1", "2", "3", "4"]:
    task = f"gittables-sp-{cv}"
    pred_file = f"./output/{task}.json"

    with open("./generate_prompt/type2id_gitsc.pkl", "rb") as file: 
        type2id = pickle.load(file)
    id2type = {int(v): k for k, v in type2id.items()}
    types = list(type2id.keys())

    with open(pred_file, "r") as f:
        col_type = json.load(f)

    ground_truth_list = []
    pred_list = []
    for i in range(len(col_type)):
        item = col_type[i]
        if item["predict"] is None:
            continue

        ground_truth = item["ground_truth"]
        pred = item["predict"].split("</s>")[0].split(", ")
        ground_truth_list.append(ground_truth)
        pred_list.append(pred)

    ts_true_list = [type2id[l[0]] for l in ground_truth_list]

    ts_pred_list = []
    for p in pred_list:
        pred_i = ', '.join(p)
        if pred_i in types:
            ts_pred_list.append(type2id[pred_i])
        else:
            ts_pred_list.append(type2id[match_closest_type(pred_i, types)])

    ts_micro_f1 = f1_score(ts_true_list, ts_pred_list, average="micro")
    ts_macro_f1 = f1_score(ts_true_list, ts_pred_list, average="macro")
    
    micro_f1_scores.append(ts_micro_f1)
    macro_f1_scores.append(ts_macro_f1)
    print(f"CV {cv}: ts_micro_f1={ts_micro_f1:.4f}, ts_macro_f1={ts_macro_f1:.4f}")

# 计算均值和标准差
avg_micro_f1 = statistics.mean(micro_f1_scores)
std_micro_f1 = statistics.stdev(micro_f1_scores)

avg_macro_f1 = statistics.mean(macro_f1_scores)
std_macro_f1 = statistics.stdev(macro_f1_scores)

# 转换为百分制并保留四位小数
avg_micro_f1_percentage = avg_micro_f1 * 100
std_micro_f1_percentage = std_micro_f1 * 100

avg_macro_f1_percentage = avg_macro_f1 * 100
std_macro_f1_percentage = std_macro_f1 * 100

# 输出结果
print(f"Average ts_micro_f1 (Percentage)={avg_micro_f1_percentage:.2f}%, Standard Deviation ts_micro_f1 (Percentage)={std_micro_f1_percentage:.4f}%")
print(f"Average ts_macro_f1 (Percentage)={avg_macro_f1_percentage:.2f}%, Standard Deviation ts_macro_f1 (Percentage)={std_macro_f1_percentage:.4f}%")

CV 0: ts_micro_f1=0.1266, ts_macro_f1=0.1054
CV 1: ts_micro_f1=0.1362, ts_macro_f1=0.1277
CV 2: ts_micro_f1=0.1234, ts_macro_f1=0.1013
CV 3: ts_micro_f1=0.1274, ts_macro_f1=0.1351
CV 4: ts_micro_f1=0.1111, ts_macro_f1=0.1005
Average ts_micro_f1 (Percentage)=12.49%, Standard Deviation ts_micro_f1 (Percentage)=0.9088%
Average ts_macro_f1 (Percentage)=11.40%, Standard Deviation ts_macro_f1 (Percentage)=1.6207%


# data analysis - TURL

In [27]:
from rapidfuzz import process, fuzz
import json
import pickle
from sklearn.metrics import multilabel_confusion_matrix, f1_score

def match_closest_type(llm_output, valid_types):
    match, score, _ = process.extractOne(llm_output, valid_types, scorer=fuzz.ratio)
    return match   # Adjust threshold as needed

In [35]:
task = f"turl-cpa"
pred_file = f"./output/{task}.json"
with open(f"./generate_prompt/type_vocab_{task}.txt", "r") as file:
    id2type = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in file}
type2id = {v: k for k, v in id2type.items()}
types = list(type2id.keys())

with open(pred_file, "r") as f:
    col_type = json.load(f)

print(types)

['baseball.baseball_player.batting_stats-baseball.batting_statistics.team', 'book.author.works_written', 'time.event.locations', 'film.film.genre', 'government.politician.party-government.political_party_tenure.party', 'people.person.place_of_birth', 'film.film.release_date_s-film.film_regional_release_date.film_release_region', 'american_football.football_player.position_s', 'cvg.computer_videogame.publisher', 'people.person.spouse_s-people.marriage.spouse', 'film.actor.film-film.performance.character', 'sports.sports_team.sport', 'base.biblioness.bibs_location.country', 'sports.sports_facility.home_venue_for-sports.team_venue_relationship.team', 'award.award_nominated_work.award_nominations-award.award_nomination.ceremony', 'film.film.directed_by', 'base.culturalevent.event.entity_involved', 'music.album.artist', 'sports.sports_team.location', 'sports.sports_position.players-sports.sports_team_roster.position', 'sports.pro_athlete.teams-sports.sports_team_roster.position', 'soccer.fo

In [36]:
import numpy as np
def f1_score_multilabel(true_list, pred_list):
    conf_mat = multilabel_confusion_matrix(np.array(true_list),
                                           np.array(pred_list))
    agg_conf_mat = conf_mat.sum(axis=0)
    # Note: Pos F1
    # [[TN FP], [FN, TP]] if we consider 1 as the positive class
    p = agg_conf_mat[1, 1] / agg_conf_mat[1, :].sum()
    r = agg_conf_mat[1, 1] / agg_conf_mat[:, 1].sum()
    
    micro_f1 = 2 * p * r / (p  + r) if (p + r) > 0 else 0.
    class_p = conf_mat[:, 1, 1] /  conf_mat[:, 1, :].sum(axis=1)
    class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)
    class_f1 = np.divide(2 * (class_p * class_r), class_p + class_r,
                         out=np.zeros_like(class_p), where=(class_p + class_r) != 0)
    class_f1 = np.nan_to_num(class_f1)
    macro_f1 = class_f1.mean()
    return (micro_f1, macro_f1, class_f1, conf_mat)


In [38]:
ground_truth_list = []
pred_list = []
for i in range(len(col_type)):
    item = col_type[i]
    if item["predict"] == None:
        continue

    ground_truth = item["ground_truth"]
    pred = item["predict"].split("</s>")[0].split(", ")

    true_vector = [0] * 121
    pred_vector = [0] * 121

    for label in ground_truth:
        true_vector[type2id[label]] = 1  # 假设 type2id[label] 返回类别索引

    for label in pred:
        if label in types:
            pred_vector[type2id[label]] = 1
        else:
            pred_vector[type2id[match_closest_type(label, types)]] = 1

    ground_truth_list.append(true_vector)
    pred_list.append(pred_vector)
                                                  

ts_micro_f1, ts_macro_f1, ts_class_f1, ts_conf_mat = f1_score_multilabel(ground_truth_list, pred_list)
print(f"micro_f1={ts_micro_f1:.4f}, macro_f1={ts_macro_f1:.4f}")


micro_f1=0.9262, macro_f1=0.8479


/tmp/ipykernel_389775/1523030160.py:12: RuntimeWarning: invalid value encountered in divide
  class_p = conf_mat[:, 1, 1] /  conf_mat[:, 1, :].sum(axis=1)
/tmp/ipykernel_389775/1523030160.py:13: RuntimeWarning: invalid value encountered in divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)
